In [3]:
import autograd
import numpy as np 
import scipy.stats as st
from autograd import numpy as np_grad
from autograd.scipy import stats as st_grad
import numpy as np

Think about evaluation Greeks for derivatives:

If the pricing is analytical, the situation is straigt-forward: differentiate the function of a price and implement the partial derivative into code. With the use of numerical methods, one can even skip the manual differentiation part

For Monte-Carlo pricing, everything is more complicated. The first idea that may come to mind the the following algoritm:
1) Calculate Monte-Carlo price of a derivative
2) Shift one risk factor by some small $h$
3) Calculate new Monte-Carlo price
4) Use numerical differentiation technique to determine the slope of a function

Problem: both PVs generated by Monte-Carlo are essentially random variables. They get closer to the true value, but are still stochastic. Therefore, the slope between them would have large variance (far larger than two PVs by themselves). Actually, variance of the partial derivative evaluated this way will be:

$$
Var(\Delta) = Var\left(\frac{\hat{C}_n(\theta_0 + h) - \hat{C}_n(\theta_0)}{\bar{h}}\right) = \frac{1}{\bar{h}^2} Var(\hat{C}_n(\theta_0 + h) - \hat{C}_n(\theta_0)) = \frac{1}{\bar{h}^2} (Var(\hat{C}_n(\theta_0 + h)) + Var(\hat{C}_n(\theta_0))) = \frac{2\sigma^2}{n\bar{h}^2} $$

Therefore, $Var(\Delta)$ depends:

* Positively on $\sigma$
* Negatively on $n$ (more simulations = higher precision, just like in regular Monte-Carlo)
* Negatively on $\bar{h}$ (contradicts traditional calculus, in which lower shift positively affects the accuracy of the derivative estimation)

However, higher $h$ increases bias of the simulated sensetivity $\Rightarrow$ Bias-Variance Tradeoff

=======================================================================================================================

But there is another way: it comes from Machine Learning and by essense is similar to the back-propogation: using chain rule to evaluate the derivative of the output to one of the inputs. It is called **Automatic Adjoint Differentiation**.

In [4]:
# before we move to the full scale Monte-Carlo approach, lets start with something simple: evaluating Greeks without explicitly defining them

class call_option:

    def __init__(self, spot, strike, T, vol, r):
        self.params =  np.array([spot, strike, T, vol, r])

    def price(self, params):
        if params.all() == params.all():
            d1 = (np_grad.log(params[0] / params[1]) + (params[4] + params[3] ** 2 / 2) * params[2]) / (np_grad.sqrt(params[2]) * params[3])
            d2 = d1 - np_grad.sqrt(params[2]) * params[3]
            return params[0] * st_grad.norm.cdf(d1) - params[1] * np_grad.exp(-params[4] * params[2]) * st_grad.norm.cdf(d2)
        else:
            d1 = (np_grad.log(self.params[0] / self.params[1]) + (self.params[4] + self.params[3] ** 2 / 2) * self.params[2]) / (np_grad.sqrt(self.params[2]) * self.params[3])
            d2 = d1 - np_grad.sqrt(self.params[2]) * self.params[3]
            return self.params[0] * st_grad.norm.cdf(d1) - self.params[1] * np_grad.exp(-self.params[4] * self.params[2]) * st_grad.norm.cdf(d2)
    
    def delta(self):
        d1 = (np.log(self.params[0] / self.params[1]) + (self.params[4] + self.params[3] ** 2 / 2) * self.params[2]) / (np.sqrt(self.params[2]) * self.params[3])
        return st.norm.cdf(d1)
    
    def aad_first_order_greeks(self):
        nabla_f = autograd.grad(self.price)
        greeks = nabla_f(self.params)
        # print(greeks)
        print(f"Delta of an option is equal to: {np.round(greeks[0], 4)}")
        print(f"Theta of an option is equal to: {-np.round(greeks[2] / 252, 4)}")
        print(f"Vega of an option is equal to: {np.round(greeks[3], 4)}")
        print(f"Rho of an option is equal to: {np.round(greeks[4], 4)}")
    
    def aad_second_order_greeks(self):
        hess_f = autograd.hessian(self.price)
        greeks = hess_f(self.params)
        # print(greeks)
        print(f"Gamma of an option is equal to: {np.round(greeks[0][0], 4)}")
        print(f"Vanna of an option is equal to: {np.round(greeks[0][3], 4)}")
        print(f"Volga of an option is equal to: {np.round(greeks[3][3], 4)}")
        print(f"Charn of an option is equal to: {-np.round(greeks[0][2], 4)}")

In [5]:
call_option(100.0, 110.0, 1.0, 0.2, 0.05).aad_first_order_greeks()

Delta of an option is equal to: 0.4496
Theta of an option is equal to: -0.0234
Vega of an option is equal to: 39.576
Rho of an option is equal to: 38.9247


In [6]:
call_option(100.0, 110.0, 1.0, 0.2, 0.05).aad_second_order_greeks()

Gamma of an option is equal to: 0.0198
Vanna of an option is equal to: 0.6462
Volga of an option is equal to: 8.1775
Charn of an option is equal to: -0.1636


In [7]:
# before we move to the full scale Monte-Carlo approach, lets start with something simple: evaluating Greeks without explicitly defining them

class call_option:

    def __init__(self, spot, strike, T, vol, r, N = 0):
        self.params =  np.array([spot, strike, T, vol, r])
        self.N = N

    def price(self, params = []):
        if len(params) != 0:
            d1 = (np_grad.log(params[0] / params[1]) + (params[4] + params[3] ** 2 / 2) * params[2]) / (np_grad.sqrt(params[2]) * params[3])
            d2 = d1 - np_grad.sqrt(params[2]) * params[3]
            return params[0] * st_grad.norm.cdf(d1) - params[1] * np_grad.exp(-params[4] * params[2]) * st_grad.norm.cdf(d2)
        else:
            d1 = (np_grad.log(self.params[0] / self.params[1]) + (self.params[4] + self.params[3] ** 2 / 2) * self.params[2]) / (np_grad.sqrt(self.params[2]) * self.params[3])
            d2 = d1 - np_grad.sqrt(self.params[2]) * self.params[3]
            return self.params[0] * st_grad.norm.cdf(d1) - self.params[1] * np_grad.exp(-self.params[4] * self.params[2]) * st_grad.norm.cdf(d2)
    
    def delta(self):
        d1 = (np.log(self.params[0] / self.params[1]) + (self.params[4] + self.params[3] ** 2 / 2) * self.params[2]) / (np.sqrt(self.params[2]) * self.params[3])
        return st.norm.cdf(d1)
    
    def aad_first_order_greeks(self):
        nabla_f = autograd.grad(self.price)
        greeks = nabla_f(self.params)
        # print(greeks)
        print(f"Delta of an option is equal to: {np.round(greeks[0], 4)}")
        print(f"Theta of an option is equal to: {-np.round(greeks[2] / 252, 4)}")
        print(f"Vega of an option is equal to: {np.round(greeks[3], 4)}")
        print(f"Rho of an option is equal to: {np.round(greeks[4], 4)}")
    
    def aad_second_order_greeks(self):
        hess_f = autograd.hessian(self.price)
        greeks = hess_f(self.params)
        # print(greeks)
        print(f"Gamma of an option is equal to: {np.round(greeks[0][0], 4)}")
        print(f"Vanna of an option is equal to: {np.round(greeks[0][3], 4)}")
        print(f"Volga of an option is equal to: {np.round(greeks[3][3], 4)}")
        print(f"Charn of an option is equal to: {-np.round(greeks[0][2], 4)}")

    def price_monte_carlo(self, params = []):

        if self.N == 0:
            raise ValueError("Enter positive number of paths")

        if len(params) != 0:
            spot, strike, T, vol, r = params[0], params[1], params[2], params[3], params[4]
        else:
            spot, strike, T, vol, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]

        arr = st.norm.rvs(size = (int(np_grad.round(T * 252)), self.N))

        drift_dt = (r - vol ** 2 / 2) / 252
        vol_dt = vol / np_grad.sqrt(252)
        
        paths = arr * (vol_dt) + drift_dt
        paths = spot * np_grad.exp(np_grad.cumsum(paths, axis = 0))
        return (np_grad.fmax(paths[-1, :] - strike, 0) / np_grad.exp(r * T)).mean()
        
    def aad_mc_first_order_greeks(self):
        greeks = autograd.elementwise_grad(self.price_monte_carlo)(self.params)
        print(f"Delta of an option is equal to: {np.round(greeks[0], 4)}")
        print(f"Theta of an option is equal to: {-np.round(greeks[2] / 252, 4)}")
        print(f"Vega of an option is equal to: {np.round(greeks[3], 4)}")
        print(f"Rho of an option is equal to: {np.round(greeks[4], 4)}")
    
    def aad_mc_second_order_greeks(self):
        hess_f = autograd.hessian(self.price_monte_carlo)
        greeks = hess_f(self.params)
        return greeks
    
    def delta_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([(1 + h / 100) * spot, strike, vol, T, r])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * spot)
    
    def theta_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([spot, strike, vol, T * (1 - h / 100 ), r])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * T) / 252
    
    def vega_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([spot, strike, vol * (1 + h / 100), T, r])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * vol)
    
    def rho_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([spot, strike, vol, T, r * (1 + h / 100)])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * r)

In [8]:
print(call_option(100.0, 110.0, 2.0, 0.2, 0.05, 100_000).price())
print(call_option(100.0, 110.0, 2.0, 0.2, 0.05, 100_000).price_monte_carlo())

11.455455871603185
11.55994936254778


In [9]:
call_option(100.0, 110.0, 2.0, 0.2, 0.05, 100_000).aad_mc_first_order_greeks()

Delta of an option is equal to: 0.5614
Theta of an option is equal to: 0.0023
Vega of an option is equal to: 55.4929
Rho of an option is equal to: 89.4595


In [10]:
call_option(100.0, 110.0, 2.0, 0.2, 0.05, 1_000_0).aad_mc_second_order_greeks()

array([[ 0.00000000e+00,  0.00000000e+00, -2.80370323e-02,
         5.48924892e-01,  2.22044605e-16],
       [ 0.00000000e+00,  0.00000000e+00,  2.03407452e-02,
         0.00000000e+00,  8.13629806e-01],
       [-2.80370323e-02,  2.03407452e-02,  2.83110629e-02,
        -2.74462446e+00, -1.57993891e+01],
       [ 5.48924892e-01,  0.00000000e+00, -2.74462446e+00,
        -1.42751211e+01,  0.00000000e+00],
       [-3.77475828e-15,  8.13629806e-01, -1.57993891e+01,
        -3.57809236e-13, -1.78998557e+02]])

Problem arises: zero Gamma. Lets figure out why

In [11]:
class call_option_new_payoff:

    def __init__(self, spot, strike, T, vol, r, N = 0):
        self.params =  np.array([spot, strike, T, vol, r])
        self.N = N

    def price_monte_carlo(self, params = []):

        if self.N == 0:
            raise ValueError("Enter positive number of paths")

        if len(params) != 0:
            spot, strike, T, vol, r = params[0], params[1], params[2], params[3], params[4]
        else:
            spot, strike, T, vol, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]

        arr = st.norm.rvs(size = (int(np_grad.round(T * 252)), self.N))

        drift_dt = (r - vol ** 2 / 2) / 252
        vol_dt = vol / np_grad.sqrt(252)
        
        paths = arr * (vol_dt) + drift_dt
        paths = spot * np_grad.exp(np_grad.cumsum(paths, axis = 0))
        return (np_grad.log(np_grad.exp(paths[-1, :] - strike) + 1) / np_grad.exp(r * T)).mean()
    
    def aad_mc_second_order_greeks(self):
        hess_f = autograd.hessian(self.price_monte_carlo)
        greeks = hess_f(self.params)
        return greeks 

In [12]:
round(call_option_new_payoff(100.0, 110.0, 2.0, 0.2, 0.05, 100_000).aad_mc_second_order_greeks()[0][0],4)

np.float64(0.014)

In [15]:
call_option(100.0, 110.0, 2.0, 0.2, 0.05, 100_000).aad_second_order_greeks()

Gamma of an option is equal to: 0.0139
Vanna of an option is equal to: 0.2459
Volga of an option is equal to: -5.4953
Charn of an option is equal to: -0.0819


In [16]:
# furthermore, now all second order Greeks are non-zero!
call_option_new_payoff(100.0, 110.0, 1.0, 0.2, 0.05, 100_000).aad_mc_second_order_greeks()

array([[ 1.98129111e-02, -1.80172903e-02, -2.25211571e-02,
         6.41547235e-01,  1.98129111e+00],
       [-1.80172903e-02,  1.63888395e-02,  1.77549520e-02,
        -2.24015275e-01, -1.44662999e+00],
       [-2.25211571e-02,  1.77549520e-02,  1.50882912e-02,
        -1.96263942e+00, -7.98566638e+00],
       [ 6.41547235e-01, -2.24015275e-01, -1.96263942e+00,
         8.85735642e+00,  2.49019351e+01],
       [ 1.98129111e+00, -1.44662999e+00, -7.98566638e+00,
         2.49019351e+01,  1.59122113e+02]])

In [17]:
# Now, lets try something more exotic, which does not have an explicit analytical pricing. In this setup, AAD shines the most.

class lookback_call_option:

    def __init__(self, spot, strike, T, vol, r, N = 0):
        self.params =  np.array([spot, strike, T, vol, r])
        self.N = N

    def price_monte_carlo(self, params = []):

        if self.N == 0:
            raise ValueError("Enter positive number of paths")

        if len(params) != 0:
            spot, strike, T, vol, r = params[0], params[1], params[2], params[3], params[4]
        else:
            spot, strike, T, vol, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]

        arr = st.norm.rvs(size = (int(np_grad.round(T * 252)), self.N))

        drift_dt = (r - vol ** 2 / 2) / 252
        vol_dt = vol / np_grad.sqrt(252)
        
        paths = arr * (vol_dt) + drift_dt
        paths = spot * np_grad.exp(np_grad.cumsum(paths, axis = 0))
        return (np_grad.log(np_grad.exp(np_grad.max(paths, axis = 0) - strike) + 1) / np_grad.exp(r * T)).mean()
        
    def aad_mc_first_order_greeks(self):
        greeks = autograd.elementwise_grad(self.price_monte_carlo)(self.params)
        print(f"Delta of an option is equal to: {np.round(greeks[0], 4)}")
        print(f"Theta of an option is equal to: {-np.round(greeks[2] / 252, 4)}")
        print(f"Vega of an option is equal to: {np.round(greeks[3], 4)}")
        print(f"Rho of an option is equal to: {np.round(greeks[4], 4)}")
    
    def aad_mc_second_order_greeks(self):
        hess_f = autograd.hessian(self.price_monte_carlo)
        greeks = hess_f(self.params)
        return greeks
    
    def delta_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([(1 + h / 100) * spot, strike, vol, T, r])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * spot)
    
    def theta_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([spot, strike, vol, T * (1 - h / 100 ), r])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * T) / 252
    
    def vega_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([spot, strike, vol * (1 + h / 100), T, r])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * vol)
    
    def rho_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([spot, strike, vol, T, r * (1 + h / 100)])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * r)

In [19]:
lookback_call_option(100, 110, 1, 0.2, 0.05, 100_000).aad_mc_first_order_greeks()

Delta of an option is equal to: 0.7869
Theta of an option is equal to: 0.0021
Vega of an option is equal to: 77.3196
Rho of an option is equal to: 44.3979


In [24]:
lookback_call_option(100, 110, 1, 0.2, 0.05, 100_000).delta_bump(2)

np.float64(0.8007063422034602)

In [25]:
import pandas as pd
x = lookback_call_option(100, 110, 2.0, 0.2, 0.05, 100_000).aad_mc_second_order_greeks()
df = pd.DataFrame(x)
df.rename(columns = {0: "Spot", 1: "Strike", 2: "T", 3: "Vol", 4: "Rate"}, index = {0: "Spot", 1: "Strike", 2: "T", 3: "Vol", 4: "Rate"}, inplace=True)


: 

In [32]:
df

,Spot,Strike,T,Vol,Rate
Spot,9.040869e-19,-4.413742e-19,0.458978,4.589780e+00,-2.220446e-16
Strike,6.566466e-19,-3.579331e-19,0.047561,1.769417e-16,9.512294e-01
T,4.589780e-01,4.756147e-02,-12.471714,3.368768e+02,9.940347e+01
Vol,4.589780e+00,1.931226e-16,336.876772,1.073878e+03,-1.705303e-13
Rate,-4.440892e-16,9.512294e-01,99.403475,-4.060576e-14,-1.046352e+02


In [ ]:
# before we move to the full scale Monte-Carlo approach, lets start with something simple: evaluating Greeks without explicitly defining them

class asian_call_option:

    def __init__(self, spot, strike, T, vol, r, N = 0):
        self.params =  np.array([spot, strike, T, vol, r])
        self.N = N

    def price_monte_carlo(self, params = []):

        if self.N == 0:
            raise ValueError("Enter positive number of paths")

        if len(params) != 0:
            spot, strike, T, vol, r = params[0], params[1], params[2], params[3], params[4]
        else:
            spot, strike, T, vol, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]

        arr = st.norm.rvs(size = (int(np_grad.round(T * 252)), self.N))

        drift_dt = (r - vol ** 2 / 2) / 252
        vol_dt = vol / np_grad.sqrt(252)
        
        paths = arr * (vol_dt) + drift_dt
        paths = spot * np_grad.exp(np_grad.cumsum(paths, axis = 0))
        return (np_grad.log(np_grad.exp(np_grad.mean(paths, axis = 0) - strike) + 1) / np_grad.exp(r * T)).mean()
        
    def aad_mc_first_order_greeks(self):
        greeks = autograd.elementwise_grad(self.price_monte_carlo)(self.params)
        # print(f"Delta of an option is equal to: {np.round(greeks[0], 4)}")
        # print(f"Theta of an option is equal to: {-np.round(greeks[2] / 252, 4)}")
        # print(f"Vega of an option is equal to: {np.round(greeks[3], 4)}")
        # print(f"Rho of an option is equal to: {np.round(greeks[4], 4)}")
        return greeks
    
    def aad_mc_second_order_greeks(self):
        hess_f = autograd.hessian(self.price_monte_carlo)
        greeks = hess_f(self.params)
        print(f"Gamma of an option is equal to: {np.round(greeks[0][0], 4)}")
        print(f"Vanna of an option is equal to: {np.round(greeks[0][3], 4)}")
        print(f"Volga of an option is equal to: {np.round(greeks[3][3], 4)}")
        print(f"Charn of an option is equal to: {-np.round(greeks[0][2], 4)}")
        return greeks
    
    def delta_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([(1 + h / 100) * spot, strike, vol, T, r])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * spot)
    
    def theta_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([spot, strike, vol, T * (1 - h / 100 ), r])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * T) / 252
    
    def vega_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([spot, strike, vol * (1 + h / 100), T, r])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * vol)
    
    def rho_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([spot, strike, vol, T, r * (1 + h / 100)])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * r)

In [36]:
asian_call_option(100.0, 105.0, 1.0, 0.2, 0.05, 100_000).aad_mc_first_order_greeks()

Delta of an option is equal to: 0.5283
Theta of an option is equal to: -0.011
Vega of an option is equal to: 1.6364
Rho of an option is equal to: 51.9253


array([ 0.52832712, -0.49950884,  2.75990191,  1.63637204, 51.92529419])

In [1]:
asian_call_option(100.0, 105.0, 1.0, 0.2, 0.05, 100_000).aad_mc_second_order_greeks()

NameError: name 'asian_call_option' is not defined

In [47]:
# %%timeit
# mean time = 1.54 sec for 10 runs
lst = []
for i in range(100):
    lst.append(asian_call_option(100.0, 105.0, 1.0, 0.2, 0.05, 10000).delta_bump(2))

print(np.std(lst))

0.00659243493323625


In [48]:
# %%timeit
# mean time = 1.04 sec for 10 runs
lst = []
for i in range(100):
    lst.append(asian_call_option(100.0, 105.0, 1.0, 0.2, 0.05, 10000).aad_mc_first_order_greeks()[0])

print(np.std(lst))

0.0027321488245962747


Total perfomance difference for calculating first-order Greeks: $=\left(\frac{0.0066}{0.0027}\right)^2 \times \frac{1.54}{1.04} \times 4 \space Greeks \approx 35$ times AAD is more efficient than bumping, when calculating first order Greeks. For the full set of second-order Greeks I expect to see a many hundreds of times performance boost.